In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv2D, MaxPool2D
from keras.models import Model
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import weave
import json
import time

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
keras.backend.image_data_format()

'channels_first'

In [3]:
def lr_schedule(epoch):
    return (0.00100 - 7e-6*epoch)

def straight_stack(x, num_filters, rep = 3, weight_decay = 1e-4, drop = 0.2):
    for n in range(rep):
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = Conv2D((2**n)*num_filters, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(x)
        x = BatchNormalization()(x)
        x = MaxPool2D()(x)
        x = Dropout(drop)(x)
    
    return x

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [4]:
batch_size = 64
num_classes = 10
steps_per_epoch = 1
epochs = 1
num_val_test = 500

In [5]:
viewer = keras.callbacks.TensorBoard(log_dir='./logs', 
                                     histogram_freq=0, 
                                     batch_size=batch_size, 
                                     write_graph=True)

inputs = Input(shape=(3,32,32))
x = weave.cDense(inputs, 4, 2)
x = weave.cDense(x, 4, 2)
x = weave.cDense(x, 2, 2)
x = [MaxPool2D()(n) for n in x]
x = weave.cDense(x, 1, 2)

x = Flatten()(x[0])

predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=[inputs], outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
num_params = model.count_params()
print('number of parameters %d' % num_params)
    
    

('len(inputs) = %d', 1)
('Inputs', [<tf.Tensor 'input_1:0' shape=(?, 3, 32, 32) dtype=float32>])
4
('len(inputs) = %d', 4)
('Inputs', [<tf.Tensor 'cDense_layer/pyrm_weave_disjoint_not_pure_combine_unit/conv2d_3/Relu:0' shape=(?, 2, 32, 32) dtype=float32>, <tf.Tensor 'cDense_layer/pyrm_weave_disjoint_not_pure_combine_unit_1/conv2d_6/Relu:0' shape=(?, 2, 32, 32) dtype=float32>, <tf.Tensor 'cDense_layer/pyrm_weave_disjoint_not_pure_combine_unit_2/conv2d_9/Relu:0' shape=(?, 2, 32, 32) dtype=float32>, <tf.Tensor 'cDense_layer/pyrm_weave_disjoint_not_pure_combine_unit_3/conv2d_12/Relu:0' shape=(?, 2, 32, 32) dtype=float32>])
('len(inputs) = %d', 4)
('Inputs', [<tf.Tensor 'cDense_layer_1/pyrm_weave_disjoint_not_pure_combine_unit/conv2d_15/Relu:0' shape=(?, 2, 32, 32) dtype=float32>, <tf.Tensor 'cDense_layer_1/pyrm_weave_disjoint_not_pure_combine_unit_1/conv2d_18/Relu:0' shape=(?, 2, 32, 32) dtype=float32>, <tf.Tensor 'cDense_layer_1/pyrm_weave_disjoint_not_pure_combine_unit_2/conv2d_21/Relu:0

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape)
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

(50000, 3, 32, 32)


In [7]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch= steps_per_epoch,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test[0:num_val_test],y_test[0:num_val_test]),
                    callbacks=[LearningRateScheduler(lr_schedule), viewer]).history

Epoch 1/1
1/1 [==============================] - 55s 55s/step - loss: 2.5734 - acc: 0.1094 - val_loss: 2.6797 - val_acc: 0.1140


{'acc': [0.109375],
 'loss': [2.5734152793884277],
 'val_acc': [0.11400000002980232],
 'val_loss': [2.679702188491821]}